-----------------------

разбиение по категориям

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/train.csv')

In [ ]:
df_train.dtypes

sign         int64
filename    object
dtype: object

In [ ]:
df_train.head()

,sign,filename
0,0,200d8766-4240-44d1-a2da-65dc4ee7973a
1,0,87174854-b7b6-4cb2-8183-4ca2bbd26b9d
2,0,c089638e-b683-436a-a3a3-3e5be972c4f6
3,1,8790e76c-f485-49a6-abaf-3147436ca085
4,0,c06b01db-8934-4c72-b568-508a32c4b170


In [ ]:
df_true = df_train[df_train['sign']==1]

In [ ]:
len(df_true['filename'])

7899

In [ ]:
df_false = df_train[df_train['sign']==0]

In [ ]:
len(df_false['filename'])

23181

Создание директорий

In [ ]:
import os

In [ ]:
import zipfile
with zipfile.ZipFile('/content/avia-train.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/content')

In [ ]:
os.mkdir('train_data_for_avia')

In [ ]:
os.makedirs(r'/content/train_data_for_avia/1')

In [ ]:
os.makedirs(r'/content/train_data_for_avia/0')

добавление изображений по категориям в каталоги

In [ ]:
os.listdir('/content/avia-train')

['ed0bbabf-77d8-4930-9802-a3fe85c42cf6.png',
 '0623405c-3552-4404-a39e-d25e60ec4a02.png',
 '7e21e5dd-510c-4be7-8dbc-f6696d336fb4.png',
 'b817b8f0-8bb3-4454-b404-c0d111bfcd95.png',
 '0edfb3af-dec6-48f6-8a82-8b6cbfb5716f.png',
 '8a84adad-5aa9-4c31-a821-24135d128f5b.png',
 'cb437ece-4823-46b7-875e-edb2aa0bf9d2.png',
 '6b09a49b-95a7-4d7a-8369-5d7c7eb4bf7c.png',
 '0f36b4a2-06c0-4144-9ee1-b2c5eab00a36.png',
 'e1d10d7a-dd5f-4dc2-aaa5-929c7b340826.png',
 '64183948-c30a-43ca-bb43-21aac3ed501e.png',
 'f2561a9c-8fe6-44e8-8480-cbd59d621e11.png',
 '0e637409-cc3f-4ccf-9826-c55eb524c478.png',
 '7d7c361d-2b00-43bd-931f-d0e68964024a.png',
 '54b326c3-e65d-41b7-986d-5f25886f6758.png',
 '918eaf70-4de4-46af-9889-9595747901d0.png',
 'f0405305-b4dd-4916-a47a-bd355ebb4a56.png',
 '0513c1f8-73fd-4a71-8fe5-390e351e7fa5.png',
 'd16f709e-0b20-4c62-9070-e6020ea818cf.png',
 '6f9d3133-8095-4b1a-bd7a-11eeb82b0afc.png',
 '94a7a48a-a265-442b-8d01-ab0b2c39aafc.png',
 '84e96755-f9dc-4945-bddf-d4efab23ae51.png',
 '4e7448c7

In [ ]:
df_true['filename'].iloc[0]

'8790e76c-f485-49a6-abaf-3147436ca085'

In [ ]:
import shutil

In [ ]:
for i in range(len(df_true)):
  shutil.copy('/content/avia-train/{}.png'.format(df_true['filename'].iloc[i]), '/content/train_data_for_avia/1')

In [ ]:
len(os.listdir('/content/train_data_for_avia/1'))

7899

In [ ]:
for i in range(len(df_false)):
  shutil.copy('/content/avia-train/{}.png'.format(df_false['filename'].iloc[i]), '/content/train_data_for_avia/0')

In [ ]:
len(os.listdir('/content/train_data_for_avia/0'))

23181

разделение на обучающий и валидационный датасеты

In [ ]:
from tqdm import tqdm

In [ ]:
train_dir = 'train'
val_dir = 'val'

In [ ]:
class_names = ['1', '0']

In [ ]:
data_root = '/content/train_data_for_avia'

In [ ]:
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

In [ ]:
for class_name in class_names:
    source_dir = os.path.join(data_root, class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|██████████| 23181/23181 [00:02<00:00, 8781.78it/s]


обработка изображений

In [ ]:
import torchvision
from torchvision import transforms

In [ ]:
train_data_path = '/content/train_data_for_avia'

In [ ]:
train_transform = transforms.Compose([
                                transforms.Resize(224), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                     std=[0.229, 0.224, 0.225])
])

In [ ]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transform)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transform)

In [ ]:
import torch

In [ ]:
from torch import utils

In [ ]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import matplotlib.pyplot as plt

Загрузка и обработка тестовых изображений

In [ ]:
#with zipfile.ZipFile('/content/avia-test.zip', 'r') as zip_obj:
 #  # Extract all the contents of zip file in current directory
 #  zip_obj.extractall('/content')

In [ ]:
#test_data_path = '/content/avia-test'

In [ ]:
#test_dataset = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

Разработка нейронной сети

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.determinictic = True

In [ ]:
import torchvision.models as models

In [ ]:
#model = models.resnet18(pretrained=True)

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
for name, param in model.named_parameters():
  if 'bn' not in name:
      param.requires_grad = False

In [ ]:
from torch import nn

In [ ]:
model.fc = nn.Linear(model.fc.in_features,2)

In [ ]:
#model.fc =  nn.Sequential(nn.Linear(model.fc.in_features,500), nn.ReLU(),nn.Dropout(), nn.Linear(500,2))

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
train_model(model, loss, optimizer, scheduler, num_epochs=2)

NameError: ignored

In [ ]:
with zipfile.ZipFile('/content/avia-test-extended.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/content')

In [ ]:
torch.save(model.state_dict(), '/content/model_param')

In [ ]:
state_dict = torch.load('/content/model_param')

In [ ]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
test_dir = 'test'
shutil.copytree('/content/avia-test-extended', os.path.join(test_dir, 'unknown'))

'test/unknown'

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
test_dataset = ImageFolderWithPaths('/content/test', val_transform)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

  0%|          | 0/3125 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 3125/3125 [1:57:28<00:00,  2.26s/it]


In [ ]:
submission_df = pd.DataFrame.from_dict({'sign': test_predictions, 'filename': test_img_paths, })

In [ ]:
submission_df['sign'] = submission_df['sign'].map(lambda pred: 1 if pred > 0.5 else 0)
submission_df['filename'] = submission_df['filename'].str.replace('/content/test/unknown/', '')
submission_df['filename'] = submission_df['filename'].str.replace('.jpeg', '')
#submission_df.set_index('filename', inplace=True)
submission_df.head(n=6)

,sign,filename
0,0,0001ae74-0621-4612-bf01-873b4aa8ca04
1,0,000335cc-3d29-47a1-901c-b519c24a3046
2,0,00033beb-a9dc-487c-ae84-58f854ad3541
3,0,0003e529-d802-4d0f-b44c-cf581ab45233
4,0,00041fad-16d2-483f-9e57-cbccba08df30
5,0,00043035-aca1-4451-95ed-d77f7e98f287


In [ ]:
submission_df['sign'].value_counts()

0    99993
1        7
Name: sign, dtype: int64

In [ ]:
submission_df.to_csv('submission_1.csv', index=False)

In [ ]:
df_row = pd.read_csv('/content/test_extended')

In [ ]:
df_row = df_row[1000:]

In [ ]:
df_row.shape

(100000, 1)

In [ ]:
df_10000 = pd.merge(df_row, submission_df, how='left')

In [ ]:
df_10000.head()

,filename,sign
0,6668257e-4c23-4e06-8e85-bbe602ba59bd,0
1,bb09e1b3-0a77-4ce8-b2aa-b81217e98a12,0
2,2a776a60-b88a-4510-b9cc-c0cdfd0c1cb8,0
3,23dd1d1f-2a2b-4f95-b712-f03550c05f2c,0
4,7ce93897-cf09-4a56-822c-145ed62078e8,0


In [ ]:
df_10000 =df_10000.reindex(columns=['sign', 'filename'])

решения для первого набора

In [ ]:
with zipfile.ZipFile('/content/avia-test.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/content')

In [ ]:
test_dir = 'test_1'
shutil.copytree('/content/avia-test', os.path.join(test_dir, 'unknown'))

'test_1/unknown'

In [ ]:
test_dataset_1 = ImageFolderWithPaths('/content/test_1', val_transform)

In [ ]:
test_dataloader_1 = torch.utils.data.DataLoader(test_dataset_1, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
model.eval()

test_predictions_1 = []
test_img_paths_1 = []
for inputs, labels, paths in tqdm(test_dataloader_1):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions_1.append(torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths_1.extend(paths)
    
test_predictions_1 = np.concatenate(test_predictions_1)

100%|██████████| 32/32 [01:12<00:00,  2.26s/it]


In [ ]:
submission_df_1 = pd.DataFrame.from_dict({'sign': test_predictions_1, 'filename': test_img_paths_1 })

In [ ]:
submission_df_1['sign'] = submission_df_1['sign'].map(lambda pred: 1 if pred > 0.5 else 0)
submission_df_1['filename'] = submission_df_1['filename'].str.replace('/content/test_1/unknown/', '')
submission_df_1['filename'] = submission_df_1['filename'].str.replace('.png', '')
#submission_df.set_index('filename', inplace=True)
submission_df_1.head(n=6)

,sign,filename
0,0,0019eb80-8333-4218-9fc7-68aac6dd9f06
1,0,003597dc-2212-478c-8e06-bcafd17ce351
2,0,00a90c41-965e-45d0-90c2-391e20cb25b7
3,0,00dc43c9-7567-485e-be98-304473b75c23
4,0,00fe5609-d380-4135-95ac-f32b6bb4655f
5,0,0101750c-7f73-42ba-9fe3-35e9a6fbf923


In [ ]:
first_df = pd.read_csv('/content/test.csv')

In [ ]:
merged_df = pd.merge(first_df, submission_df_1, how='left')

In [ ]:
merged_df.head()

,filename,sign
0,68f33844-472b-4111-b600-f90d544833c7,0
1,7d93a21d-1f16-49ce-8fcc-edf12c40f549,0
2,4a820650-7acd-489a-ad14-9d7ad8c73b6b,0
3,819b216b-2b6c-4539-a722-70648c0706c6,1
4,45f7c47d-03cc-40cd-acc5-b8c1c57872fa,0


In [ ]:
new_df_merged = merged_df.reindex(columns=['sign', 'filename'])

In [ ]:
final_df = pd.concat([new_df_merged, df_10000], ignore_index=True)

In [ ]:
final_df

,sign,filename
0,0,68f33844-472b-4111-b600-f90d544833c7
1,0,7d93a21d-1f16-49ce-8fcc-edf12c40f549
2,0,4a820650-7acd-489a-ad14-9d7ad8c73b6b
3,1,819b216b-2b6c-4539-a722-70648c0706c6
4,0,45f7c47d-03cc-40cd-acc5-b8c1c57872fa
...,...,...
100995,0,82aba9fd-3f78-4087-99f6-af9f5a990677
100996,0,3bb91d2b-53fb-42d4-a644-7e30a39572cd
100997,0,fba3e0de-d8b7-4721-82ae-f6ff135bd746
100998,0,09fa231e-6998-4904-acd7-2ed71408296e


In [ ]:
final_df.to_csv('submission_2.csv', index=False)